<h1> 1 Installing Libraries </h1>

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from subprocess import check_output
%matplotlib inline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

import os
import gc
import json
import pickle
import glob

import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import SGDClassifier, SGDRegressor
import xgboost as xgb
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")
from subprocess import check_output
%matplotlib inline

import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from bs4 import BeautifulSoup
from fuzzywuzzy import fuzz
from sklearn.manifold import TSNE

from os import path
from PIL import Image

import time
import warnings
import sqlite3
from sqlalchemy import create_engine # database connection
import csv
import os
warnings.filterwarnings("ignore")
import datetime as dt
import numpy as np
from nltk.corpus import stopwords
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
# from sklearn.metrics.classification import accuracy_score, log_loss
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from scipy.sparse import hstack
from sklearn.multiclass import OneVsRestClassifier
# from sklearn.cross_validation import StratifiedKFold 
from collections import Counter, defaultdict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import normalized_mutual_info_score
from sklearn.ensemble import RandomForestClassifier



from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDClassifier
# from mlxtend.classifier import StackingClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_curve, auc, roc_curve

In [3]:
def load_txt(name):
    with open(name, "r") as f:
        return f.read()

def load_pickle(name):
    with open(name, "rb") as handle:
        return pickle.load(handle)
    
def dump_pickle(name, file):
    with open(name, "wb") as handle:
        pickle.dump(file, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [14]:
drug_features_data = pd.read_csv("../ix_word/final_features.csv", index_col="Unnamed: 0")
drug_features_data.head(2)

,patient_id,review_0,review_1,review_2,review_3,review_4,review_5,review_6,review_7,review_8,...,review_763,review_764,review_765,review_766,review_767,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,len_review_by_patient_unique_word,base_score
0,206461,0.059691,0.008081,0.080178,-0.266999,-0.264136,-0.360968,0.64633,0.423532,-0.074173,...,-0.065599,-0.351725,-0.171581,0.362696,-0.022024,9,20-May-12,27,10,8.022969
1,95260,-0.477708,-0.197286,-0.113292,-0.019980,-0.573079,-0.220722,0.34921,-0.051921,-0.013572,...,0.230648,-0.213165,-0.396574,0.421274,0.357504,8,27-Apr-10,192,56,7.858458


In [17]:
drug_features_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32165 entries, 0 to 32164
Columns: 774 entries, patient_id to base_score
dtypes: float64(769), int64(4), object(1)
memory usage: 190.2+ MB


In [18]:
drug_features_data_X = drug_features_data.drop(columns=["base_score"])
drug_features_data_y = drug_features_data["base_score"]
drug_features_data_X.shape, drug_features_data_y.shape

((32165, 773), (32165,))

In [19]:
train_data_X = drug_features_data_X.iloc[:10000, :].drop(columns=["patient_id", "drug_approved_by_UIC"]).values
print(train_data_X.shape)
train_data_y = drug_features_data_y.iloc[:10000].values
print(train_data_y.shape)
validate_data_X = drug_features_data_X.iloc[10000:, :].drop(columns=["patient_id", "drug_approved_by_UIC"]).values
print(validate_data_X.shape)
validate_data_y = drug_features_data_y.iloc[10000:].values
print(validate_data_y.shape)

(10000, 771)
(10000,)
(22165, 771)
(22165,)


In [20]:
drug_features_data_X.iloc[10000:, :].drop(columns=["patient_id", "drug_approved_by_UIC"])

,review_0,review_1,review_2,review_3,review_4,review_5,review_6,review_7,review_8,review_9,...,review_761,review_762,review_763,review_764,review_765,review_766,review_767,effectiveness_rating,number_of_times_prescribed,len_review_by_patient_unique_word
10000,-0.110975,0.109967,0.288496,-0.284090,-0.288065,-0.261429,0.459298,0.463319,-0.131293,-0.355930,...,-0.040490,0.437305,0.018764,-0.270523,-0.091628,0.212438,0.225514,1,15,30
10001,-0.063783,0.121525,0.233503,-0.227393,-0.148515,-0.192648,0.422375,0.445765,-0.154808,-0.205729,...,0.018606,0.264033,0.068824,-0.335408,-0.154329,0.375006,0.229754,9,26,58
10002,-0.092103,0.085791,0.179216,-0.147141,-0.253354,-0.294751,0.464901,0.548932,-0.152246,-0.135758,...,-0.199576,0.155069,0.172735,-0.341181,-0.357343,0.380748,0.200482,10,45,26
10003,-0.088123,0.143031,0.174271,-0.337862,-0.256687,-0.249824,0.385136,0.194178,-0.107842,-0.135781,...,-0.161966,0.295373,-0.030554,-0.294674,-0.187774,0.473704,0.328117,8,11,37
10004,0.055386,0.179553,0.386874,-0.169798,-0.300282,-0.164665,0.461001,0.523888,-0.228498,-0.252731,...,-0.114124,0.343070,0.005280,-0.243520,-0.122548,0.289546,0.024625,2,1,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32160,-0.025693,0.060552,0.028152,-0.303997,-0.072068,-0.020022,0.416418,0.524290,-0.104854,-0.238810,...,-0.175059,0.215841,-0.022023,-0.240635,-0.156761,0.312285,0.353882,9,89,44
32161,0.017898,0.132657,0.272135,-0.005240,-0.158938,-0.226180,0.550534,0.561894,-0.256864,-0.135150,...,-0.118178,0.221388,-0.073159,-0.322715,-0.527602,0.311378,0.064666,6,0,49
32162,-0.154145,0.190222,0.170894,-0.310281,-0.231704,-0.369669,0.512393,0.414607,-0.019496,-0.201162,...,-0.098297,0.323354,-0.077019,-0.263817,-0.255950,0.457359,0.134897,9,25,66
32163,0.004854,0.004234,0.149718,-0.234569,-0.357956,-0.022622,0.382827,0.521656,-0.264836,-0.200878,...,-0.328201,0.209254,-0.001249,-0.181040,-0.206644,0.307512,0.279487,8,22,61


In [21]:
drug_features_test = pd.read_csv("../ix_word/final_features_test.csv", index_col="Unnamed: 0")
drug_features_test

,patient_id,review_0,review_1,review_2,review_3,review_4,review_5,review_6,review_7,review_8,...,review_762,review_763,review_764,review_765,review_766,review_767,effectiveness_rating,drug_approved_by_UIC,number_of_times_prescribed,len_review_by_patient_unique_word
0,163740.0,-0.019849,0.173974,0.128156,-0.329652,-0.305466,-0.374780,0.513443,0.256115,-0.174848,...,0.380650,0.049244,-0.215225,-0.279196,0.303954,0.058522,10.0,28-Feb-12,22.0,36.0
1,39293.0,0.079807,0.090857,0.170656,-0.152019,-0.202442,-0.290995,0.381226,0.454116,-0.339259,...,0.254681,0.206014,-0.216410,-0.261225,0.255563,0.014028,9.0,5-Mar-17,35.0,62.0
2,208087.0,-0.074708,-0.016995,0.414342,-0.308373,-0.060608,-0.190645,0.583799,0.645578,-0.059241,...,0.126178,-0.066831,-0.213132,-0.483495,0.408229,0.157913,4.0,3-Jul-14,13.0,34.0
3,23295.0,-0.109779,0.178474,0.281025,-0.213470,-0.211641,-0.360548,0.450201,0.481589,-0.108589,...,0.256800,0.002749,-0.147731,-0.154975,0.328237,0.123911,7.0,18-Oct-16,21.0,34.0
4,97013.0,0.033426,0.124495,0.069546,-0.365152,-0.168140,-0.293745,0.471172,0.675921,-0.193613,...,0.355150,0.013335,-0.064397,-0.193974,0.426754,0.086814,2.0,13-Jan-15,44.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10755,151266.0,0.193067,0.047614,0.089650,-0.207552,-0.234617,-0.334094,0.480928,0.554582,0.058097,...,0.440808,-0.252605,-0.250491,-0.184165,0.331224,0.131948,1.0,11-Nov-17,2.0,19.0
10756,139347.0,-0.043599,0.081173,0.033114,-0.239584,-0.342154,-0.321097,0.805464,0.524058,-0.109156,...,0.477767,-0.064304,-0.291467,-0.474799,0.457937,0.089260,1.0,30-Sep-14,18.0,17.0
10757,159999.0,-0.103857,-0.139298,0.128744,-0.250422,0.011483,-0.036569,0.387917,0.232539,-0.136333,...,0.236612,0.049451,-0.212269,-0.290575,0.324503,0.239300,10.0,13-Sep-14,43.0,46.0
10758,130945.0,0.052184,-0.005329,0.318639,-0.251990,-0.318230,-0.132561,0.548931,0.764816,-0.309994,...,0.275493,-0.089043,-0.300018,-0.402873,0.310848,0.015512,8.0,15-Nov-10,7.0,62.0


In [24]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X = scaler_X.fit_transform(train_data_X)
y = scaler_y.fit_transform(train_data_y.reshape((10000, 1)))

In [25]:
X.shape

(10000, 771)

In [26]:
y.shape

(10000, 1)

In [27]:
validate_data_X.shape

(22165, 771)

# 1. Linear Regresson

In [28]:
lr = LinearRegression()
lr.fit(X, y)
lr_predicted_y = scaler_y.inverse_transform(lr.predict(scaler_X.transform(validate_data_X[0:100])))
print("mean sq", mean_squared_error(validate_data_y[0:100], lr_predicted_y))
print("mean_absolute_error ",mean_absolute_error(validate_data_y[0:100], lr_predicted_y))
print("r2 Score",r2_score(validate_data_y[0:100], lr_predicted_y))

mean sq 2.1707111391566642
mean_absolute_error  1.1467261272316622
r2 Score 0.07487242072947786


# 2. KNN

In [29]:
alpha = [x for x in range(1, 15, 2)]
for i in alpha:
    print("\n", i) # (n_estimators=i,random_state=42,n_jobs=-1)
    knnr=KNeighborsRegressor(n_neighbors=i)
    knnr.fit(X, y)
    knnr_predicted_y = scaler_y.inverse_transform(knnr.predict(scaler_X.transform(validate_data_X[0:100])))
    print("mean sq", mean_squared_error(validate_data_y[0:100], knnr_predicted_y))
    print("mean_absolute_error ",mean_absolute_error(validate_data_y[0:100], knnr_predicted_y))
    print("r2 Score",r2_score(validate_data_y[0:100], knnr_predicted_y))


 1
mean sq 4.329447065487688
mean_absolute_error  1.5985258462367624
r2 Score -0.8451514856235423

 3
mean sq 2.994480979344483
mean_absolute_error  1.364215558480859
r2 Score -0.27620708698664287

 5
mean sq 2.7264264910385636
mean_absolute_error  1.2814892469655115
r2 Score -0.16196590795284593

 7
mean sq 2.4881314664316005
mean_absolute_error  1.214184111054083
r2 Score -0.060407807803004365

 9
mean sq 2.527362642396283
mean_absolute_error  1.2033497844252132
r2 Score -0.07712760169794097

 11
mean sq 2.5385830974131034
mean_absolute_error  1.2146810669108754
r2 Score -0.0819096070973595

 13
mean sq 2.5566681187856304
mean_absolute_error  1.2270094831523608
r2 Score -0.08961719736195906


In [43]:
knnr=KNeighborsRegressor(n_neighbors=11)
knnr.fit(X, y)
knnr_predicted_y = scaler_y.inverse_transform(knnr.predict(scaler_X.transform(validate_data_X[0:100])))
print("mean sq", mean_squared_error(validate_data_y[0:100], knnr_predicted_y))
print("mean_absolute_error ",mean_absolute_error(validate_data_y[0:100], knnr_predicted_y))
print("r2 Score",r2_score(validate_data_y[0:100], knnr_predicted_y))

mean sq 2.5385830974131034
mean_absolute_error  1.2146810669108754
r2 Score -0.0819096070973595


# 3. SVR

In [152]:
alpha = [10 ** x for x in range(-5, 2)]
# class sklearn.svm.SVR(*, kernel='rbf', degree=3, gamma='scale', coef0=0.0, tol=0.001, C=1.0, epsilon=0.1, shrinking=True, cache_size=200, verbose=False, max_iter=-1)[source]
for i in alpha:
    print("\n", i)
    #SVR(kernel="rbf")
    svr = SVR(C=i) # sgr
    svr.fit(X, y)
    svr_predicted_y = scaler_y.inverse_transform(svr.predict(scaler_X.transform(validate_data_X[0:100])))
    print("mean sq", mean_squared_error(validate_data_y[0:100], svr_predicted_y))
    print("mean_absolute_error ",mean_absolute_error(validate_data_y[0:100], svr_predicted_y))
    print("r2 Score",r2_score(validate_data_y[0:100], svr_predicted_y))


 1e-05
mean sq 2.3499580378526215
mean_absolute_error  1.1443002438306953
r2 Score -0.001520170846186053

 0.0001
mean sq 2.3497866892192123
mean_absolute_error  1.1442051612135444
r2 Score -0.0014471443879080503

 0.001
mean sq 2.3487052567950446
mean_absolute_error  1.1434036858031171
r2 Score -0.0009862525895163454

 0.01
mean sq 2.3393147311591336
mean_absolute_error  1.1401344838885483
r2 Score 0.0030158617834734613

 0.1
mean sq 2.278166118219018
mean_absolute_error  1.1350354780311986
r2 Score 0.029076569375832917

 1
mean sq 2.2351590364131892
mean_absolute_error  1.1429412786221533
r2 Score 0.0474056030113138

 10
mean sq 2.2916230652344676
mean_absolute_error  1.1904228827579264
r2 Score 0.023341401489049396


In [44]:
svr = SVR(C=1) # sgr
svr.fit(X, y)
svr_predicted_y = scaler_y.inverse_transform(svr.predict(scaler_X.transform(validate_data_X[0:100])))
print("mean sq", mean_squared_error(validate_data_y[0:100], svr_predicted_y))
print("mean_absolute_error ",mean_absolute_error(validate_data_y[0:100], svr_predicted_y))
print("r2 Score",r2_score(validate_data_y[0:100], svr_predicted_y))

mean sq 2.2451607178480204
mean_absolute_error  1.1466940857061718
r2 Score 0.04314302234476097


# 4. SGDRegressor

In [155]:
alpha = [10 ** x for x in range(-5, 2)]
for i in alpha:
    print("\n", i)
    sgdr = SGDRegressor(alpha=i, penalty='elasticnet', random_state=42) # sgr
    sgdr.fit(X, y)
    sgdr_predicted_y = scaler_y.inverse_transform(sgdr.predict(scaler_X.transform(validate_data_X[0:100])))
    print("mean sq", mean_squared_error(validate_data_y[0:100], sgdr_predicted_y))
    print("mean_absolute_error ",mean_absolute_error(validate_data_y[0:100], sgdr_predicted_y))
    print("r2 Score",r2_score(validate_data_y[0:100], sgdr_predicted_y))


 1e-05
mean sq 2.5150702500183377
mean_absolute_error  1.2273745900853177
r2 Score -0.07188875116692572

 0.0001
mean sq 2.491717238061807
mean_absolute_error  1.2232200519607797
r2 Score -0.06193601492749501

 0.001
mean sq 2.358665988176339
mean_absolute_error  1.2169961292505749
r2 Score -0.005231380899919369

 0.01
mean sq 2.0220517354267913
mean_absolute_error  1.1116301378651092
r2 Score 0.13822903775123774

 0.1
mean sq 1.9506783753459789
mean_absolute_error  1.0736283675725724
r2 Score 0.1686473935816768

 1
mean sq 2.117211306498641
mean_absolute_error  1.0969204081293555
r2 Score 0.097673322141687

 10
mean sq 2.3480805640148663
mean_absolute_error  1.1561472857234678
r2 Score -0.0007200170185603927


In [45]:
sgdr = SGDRegressor(alpha=0.1, penalty='elasticnet', random_state=42) # sgr
sgdr.fit(X, y)
sgdr_predicted_y = scaler_y.inverse_transform(sgdr.predict(scaler_X.transform(validate_data_X[0:100])))
print("mean sq", mean_squared_error(validate_data_y[0:100], sgdr_predicted_y))
print("mean_absolute_error ",mean_absolute_error(validate_data_y[0:100], sgdr_predicted_y))
print("r2 Score",r2_score(validate_data_y[0:100], sgdr_predicted_y))

mean sq 1.9574448594817682
mean_absolute_error  1.0735593301780448
r2 Score 0.16576361002531326


# 5. Using XgBoost Regressor

In [46]:
xgb_model = xgb.XGBRegressor(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=3,
 min_child_weight=3,
 gamma=0,
 subsample=0.8,
 reg_alpha=200, reg_lambda=200,
 colsample_bytree=0.8,nthread=4)
print(xgb_model.fit(X, y))
xgb_predicted_y = scaler_y.inverse_transform(xgb_model.predict(scaler_X.transform(validate_data_X[0:100])))
print("mean sq", mean_squared_error(validate_data_y[0:100], xgb_predicted_y))
print("mean_absolute_error ",mean_absolute_error(validate_data_y[0:100], xgb_predicted_y))
print("r2 Score",r2_score(validate_data_y[0:100], xgb_predicted_y))

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.8,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=3, max_leaves=0, min_child_weight=3, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=4, nthread=4,
             num_parallel_tree=1, predictor='auto', ...)
mean sq 0.7748893735510621
mean_absolute_error  0.6714501039087925
r2 Score 0.6697526826926152


# 6. DTR

In [158]:
# DecisionTreeRegressor(criterion='squared_error', splitter='best', max_depth=None, min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features=None, random_state=None, max_leaf_nodes=None, min_impurity_decrease=0.0, ccp_alpha=0.0)[source])
min_samples_split = [100, 150, 200, 250]
for i in min_samples_split:
    print("\n", i)
    dtr=DecisionTreeRegressor(min_samples_split=i, random_state = 42)
    dtr.fit(X, y)
    dtr_predicted_y = scaler_y.inverse_transform(dtr.predict(scaler_X.transform(validate_data_X[0:100])))
    print("mean sq", mean_squared_error(validate_data_y[0:100], dtr_predicted_y))
    print("mean_absolute_error ",mean_absolute_error(validate_data_y[0:100], dtr_predicted_y))
    print("r2 Score",r2_score(validate_data_y[0:100], dtr_predicted_y))


 100
mean sq 0.5913115373736377
mean_absolute_error  0.36744842301391933
r2 Score 0.7479910609489329

 150
mean sq 0.5517652896850301
mean_absolute_error  0.4157228406095355
r2 Score 0.7648451341295808

 200
mean sq 0.5592596268618152
mean_absolute_error  0.4550844680127113
r2 Score 0.761651149501442

 250
mean sq 0.5820294330858019
mean_absolute_error  0.4928606615168292
r2 Score 0.7519469676172325


In [47]:
dtr=DecisionTreeRegressor(min_samples_split=150, random_state = 42)
dtr.fit(X, y)
dtr_predicted_y = scaler_y.inverse_transform(dtr.predict(scaler_X.transform(validate_data_X[0:100])))
print("mean sq", mean_squared_error(validate_data_y[0:100], dtr_predicted_y))
print("mean_absolute_error ",mean_absolute_error(validate_data_y[0:100], dtr_predicted_y))
print("r2 Score",r2_score(validate_data_y[0:100], dtr_predicted_y))

mean sq 0.5517652896850302
mean_absolute_error  0.4157228406095355
r2 Score 0.7648451341295808


# 7. RFR

In [160]:
alpha=[5, 10,20, 30, 50,100,200, 500]
for i in alpha:
    print("\n", i) # (n_estimators=i,random_state=42,n_jobs=-1)
    rfr = RandomForestRegressor(n_estimators=i, random_state=42,n_jobs=-1) # sgr
    rfr.fit(X, y)
    rfr_predicted_y = scaler_y.inverse_transform(rfr.predict(scaler_X.transform(validate_data_X[0:100])))
    print("mean sq", mean_squared_error(validate_data_y[0:100], rfr_predicted_y))
    print("mean_absolute_error ",mean_absolute_error(validate_data_y[0:100], rfr_predicted_y))
    print("r2 Score",r2_score(validate_data_y[0:100], rfr_predicted_y))


 5
mean sq 0.5028709609490535
mean_absolute_error  0.3006958020439791
r2 Score 0.7856832323765657

 10
mean sq 0.4271797841551607
mean_absolute_error  0.2848239857967475
r2 Score 0.8179417829945332

 20
mean sq 0.46982041593831264
mean_absolute_error  0.29068969196987904
r2 Score 0.7997689253772646

 30
mean sq 0.48307947577283644
mean_absolute_error  0.2955222414153172
r2 Score 0.7941180943169507

 50
mean sq 0.48584461317786465
mean_absolute_error  0.29245159604563165
r2 Score 0.7929396303436843

 100
mean sq 0.48656248371695776
mean_absolute_error  0.2913869942076425
r2 Score 0.7926336836785196

 200
mean sq 0.46004903605752717
mean_absolute_error  0.28406559037462187
r2 Score 0.8039333546521592

 500
mean sq 0.4491255465001951
mean_absolute_error  0.2805922923744762
r2 Score 0.8085887974096362


In [47]:
rfr = RandomForestRegressor(n_estimators=10, random_state=42,n_jobs=-1) # sgr
rfr.fit(X, y)
rfr_predicted_y = scaler_y.inverse_transform(rfr.predict(scaler_X.transform(validate_data_X[0:100])))
print("mean sq", mean_squared_error(validate_data_y[0:100], rfr_predicted_y))
print("mean_absolute_error ",mean_absolute_error(validate_data_y[0:100], rfr_predicted_y))
print("r2 Score",r2_score(validate_data_y[0:100], rfr_predicted_y))

mean sq 0.4301289078699712
mean_absolute_error  0.2837087553880072
r2 Score 0.8166849065571129


## Mean, Absolute Error and R2 Score on Validation Data

In [49]:
print("\t\tMean, Absolute Error and R2 Score on Validation Data")
print("\t", "-"*70)
print("\t\t\t\t1. LINEAR Regression")
print("mean sq", mean_squared_error(validate_data_y[0:100], lr_predicted_y))
print("mean_absolute_error ",mean_absolute_error(validate_data_y[0:100], lr_predicted_y))
print("r2 Score",r2_score(validate_data_y[0:100], lr_predicted_y))

print("\n\t\t\t\t2. KNN")
print("mean sq", mean_squared_error(validate_data_y[0:100], knnr_predicted_y))
print("mean_absolute_error ",mean_absolute_error(validate_data_y[0:100], knnr_predicted_y))
print("r2 Score",r2_score(validate_data_y[0:100], knnr_predicted_y))

print("\n\t\t\t\t3. SVR")
print("mean sq", mean_squared_error(validate_data_y[0:100], svr_predicted_y))
print("mean_absolute_error ",mean_absolute_error(validate_data_y[0:100], svr_predicted_y))
print("r2 Score",r2_score(validate_data_y[0:100], svr_predicted_y))

print("\n\t\t\t\t4. SGD-Regression")
print("mean sq", mean_squared_error(validate_data_y[0:100], sgdr_predicted_y))
print("mean_absolute_error ",mean_absolute_error(validate_data_y[0:100], sgdr_predicted_y))
print("r2 Score",r2_score(validate_data_y[0:100], sgdr_predicted_y))

print("\n\t\t\t\t5. XGBoost")
print("mean sq", mean_squared_error(validate_data_y[0:100], xgb_predicted_y))
print("mean_absolute_error ",mean_absolute_error(validate_data_y[0:100], xgb_predicted_y))
print("r2 Score",r2_score(validate_data_y[0:100], xgb_predicted_y))

print("\n\t\t\t\t6. DTR")
print("mean sq", mean_squared_error(validate_data_y[0:100], dtr_predicted_y))
print("mean_absolute_error ",mean_absolute_error(validate_data_y[0:100], dtr_predicted_y))
print("r2 Score",r2_score(validate_data_y[0:100], dtr_predicted_y))

print("\n\t\t\t\t7. Random Forest")
print("mean sq", mean_squared_error(validate_data_y[0:100], rfr_predicted_y))
print("mean_absolute_error ",mean_absolute_error(validate_data_y[0:100], rfr_predicted_y))
print("r2 Score",r2_score(validate_data_y[0:100], rfr_predicted_y))

		Mean, Absolute Error and R2 Score on Validation Data
	 ----------------------------------------------------------------------
				1. LINEAR Regression
mean sq 2.1707111391566642
mean_absolute_error  1.1467261272316622
r2 Score 0.07487242072947786

				2. KNN
mean sq 2.5385830974131034
mean_absolute_error  1.2146810669108754
r2 Score -0.0819096070973595

				3. SVR
mean sq 2.2451607178480204
mean_absolute_error  1.1466940857061718
r2 Score 0.04314302234476097

				4. SGD-Regression
mean sq 1.9574448594817682
mean_absolute_error  1.0735593301780448
r2 Score 0.16576361002531326

				5. XGBoost
mean sq 0.7748893735510621
mean_absolute_error  0.6714501039087925
r2 Score 0.6697526826926152

				6. DTR
mean sq 0.5517652896850302
mean_absolute_error  0.4157228406095355
r2 Score 0.7648451341295808

				7. Random Forest
mean sq 0.4301289078699712
mean_absolute_error  0.28370875538800727
r2 Score 0.8166849065571129


<h3>Here We can See That the Error of Random Forest it Very low and and R2 Score value is close to 1.</h3>

In [34]:
for i in range(100):
    print(i, validate_data_y[i], rfr_predicted_y[i])

0 6.71052568386338 6.279945117708794
1 6.765825685915322 6.765825685915322
2 6.831298989280017 6.831298989280017
3 6.562115937095545 6.638482566963994
4 6.169162419744095 6.371831256791971
5 4.821912068610578 6.137303500250596
6 8.753452412523746 8.753452412523746
7 6.1731344641759245 6.1731344641759245
8 8.656796740464273 8.656796740464273
9 5.771316867725653 5.771316867725653
10 8.022968937262847 8.022968937262847
11 7.277927183306573 7.277927183306573
12 6.471278979163006 6.471278979163006
13 8.022968937262847 8.022968937262847
14 7.900790082378434 7.900790082378434
15 5.707023638890452 5.850803925551688
16 3.167946225052172 3.167946225052173
17 5.707023638890452 5.93363289142937
18 7.403949990077454 5.7306142346404245
19 8.457255146738312 8.457255146738312
20 4.924349529579458 7.134168483525658
21 5.918437868438534 6.332011344035358
22 8.753452412523746 8.753452412523746
23 2.761670718257794 4.2306981252924984
24 8.37458833025563 8.37458833025563
25 7.19431641656822 7.1943164165682

# Test Data

In [42]:
drug_features_test = pd.read_csv("../ix_word/final_features_test.csv", index_col="Unnamed: 0")
drug_features_test = drug_features_test.drop(columns=["patient_id", "drug_approved_by_UIC"]).values

In [43]:
validate_data_X.shape

(22165, 771)

In [44]:
drug_features_test.shape

(10760, 771)

In [45]:
drug_test_data = pd.read_csv(r"C:\Users\user\OneDrive\Documents\Neuranceai\dataset\test.csv")
drug_test_data.head(2)  

,patient_id,name_of_drug,review_by_patient,drug_approved_by_UIC,number_of_times_prescribed,use_case_for_drug,effectiveness_rating
0,163740,Mirtazapine,"""I&#039;ve tried a few antidepressants over th...",28-Feb-12,22,Depression,10
1,39293,Contrave,"""Contrave combines drugs that were used for al...",5-Mar-17,35,Weight Loss,9


In [48]:
rfr_predicted_y_test_data = []
for i in range(drug_features_test.shape[0]):
#     x = scaler_y.inverse_transform(rfr.predict(scaler_X.transform([drug_features_test[i]])))[0]
    x = np.array(pd.Series(drug_features_test[i]).fillna(6.53530508e-02))
    x = scaler_y.inverse_transform(rfr.predict(scaler_X.transform([x])))[0]
    print(i, x)
    rfr_predicted_y_test_data.append(x)

0 8.757200984608478
1 8.745036833474394
2 6.162046788535377
3 4.564405615373438
4 7.596751053443429
5 7.475096634526175
6 8.358950719271759
7 6.127475169414972
8 2.2348399436105675
9 6.485994130094867
10 6.272837815756015
11 3.183955536934048
12 8.358950719271759
13 4.884977122534998
14 6.607445106972985
15 4.393288924957714
16 4.57950262578005
17 6.158768971280095
18 5.153091291558044
19 7.5203953765897715
20 7.170997223562014
21 7.403310980302783
22 7.404335807253393
23 8.757200984608478
24 6.35504055965443
25 6.109306553878856
26 7.369878171891573
27 7.267155750109901
28 6.7202549470746415
29 7.0500866311617765
30 6.245772451546388
31 3.054420106447376
32 6.355865537958907
33 8.074493292790843
34 6.504566421367798
35 6.71653435937951
36 7.580521262234863
37 8.745991303284768
38 8.753452412523746
39 6.607445106972985
40 8.31464585548871
41 7.544492890288928
42 6.819911580136614
43 6.1234614690910565
44 8.743005258264253
45 7.429046995870051
46 8.842186219376321
47 5.1337406888149335


381 4.720716430854761
382 8.233307958262657
383 7.277927183306573
384 8.648336695522234
385 3.53517649415556
386 7.277927183306573
387 7.163499314456653
388 4.319273404069607
389 8.842186219376321
390 6.6481230602742265
391 6.770978433724751
392 6.370760113378997
393 7.544492890288928
394 7.1234630765226985
395 7.250954978080509
396 3.3477247532480416
397 4.4152596781425215
398 3.9166495911168075
399 4.821588697606812
400 0.899075571349413
401 8.7151200080925
402 5.406917687876335
403 4.881685820900687
404 5.0089606168144964
405 5.3491754405736405
406 6.653815670753763
407 7.254862780164628
408 8.574485878929147
409 7.475096634526175
410 6.355865537958907
411 8.215482906027418
412 4.959338429830408
413 6.920059185010526
414 6.124057251494149
415 6.493653305451713
416 6.476183149656864
417 6.6773219728004785
418 5.560317487078089
419 8.682416473696875
420 5.653549612070854
421 7.816529783380902
422 8.574485878929147
423 8.482883456649951
424 6.677794037924427
425 7.938645770473596
426 8

756 8.074493292790843
757 5.771316867725653
758 6.207302708748353
759 6.76561244348505
760 8.823947260422257
761 4.597133439396204
762 6.208646427113182
763 6.622680224569159
764 8.563274491655593
765 5.538040554795084
766 6.493653305451713
767 7.614215701636045
768 5.433997586477339
769 6.5114336812428935
770 5.691791949528471
771 7.0500866311617765
772 8.648336695522234
773 7.477697851583277
774 6.653815670753763
775 8.805914067059051
776 8.245434140579743
777 4.392806886609837
778 6.562115937095545
779 7.914369886731474
780 7.921906947449914
781 6.935684111767527
782 7.0500866311617765
783 8.469379704137364
784 6.515511802414823
785 7.839952234736723
786 8.815954400496096
787 7.106682637574135
788 8.745991303284768
789 6.471278979163006
790 8.571013126730215
791 6.109306553878856
792 4.839663719805017
793 5.835108244339466
794 3.53517649415556
795 5.687366188037061
796 3.167946225052173
797 4.7082993567853855
798 7.170997223562014
799 6.765825685915322
800 7.648306726574672
801 4.93

1127 8.574485878929147
1128 8.745991303284768
1129 2.195434264722608
1130 6.279229488091716
1131 7.805273298557728
1132 6.137547834215492
1133 6.71302127922257
1134 7.475096634526175
1135 7.103936881497545
1136 7.277927183306573
1137 5.793133906306418
1138 8.001996457056773
1139 6.3732589822241685
1140 6.935684111767527
1141 4.884977122534998
1142 6.607445106972985
1143 8.805914067059051
1144 8.074493292790843
1145 8.31464585548871
1146 8.358345332641477
1147 7.170997223562014
1148 6.920059185010526
1149 6.21564611335867
1150 7.084225295181232
1151 7.984264865636664
1152 7.19431641656822
1153 7.53399454284663
1154 6.165484183173448
1155 7.312774019149931
1156 7.079782698942971
1157 7.19431641656822
1158 6.362434680168343
1159 6.471278979163006
1160 5.429024259791327
1161 6.762245042844588
1162 8.174543577159849
1163 6.127475169414972
1164 7.736175873181428
1165 8.31464585548871
1166 8.358950719271759
1167 6.548256302136768
1168 7.15578397834662
1169 6.383855104378132
1170 7.19431641656

1504 6.287576175419695
1505 8.670935366307415
1506 8.246677534625743
1507 6.519118401659605
1508 7.839952234736723
1509 6.8149381584299284
1510 6.548256302136768
1511 6.35504055965443
1512 7.067359716161593
1513 6.608813645712799
1514 6.562115937095545
1515 7.402296540208344
1516 7.736175873181428
1517 7.249504825573507
1518 8.7151200080925
1519 6.124057251494149
1520 7.837632027405768
1521 8.482883456649951
1522 5.9449003905608375
1523 6.1731344641759245
1524 7.102481680145139
1525 6.191610618356124
1526 6.109306553878856
1527 8.580480617777244
1528 6.493653305451713
1529 6.853353475041125
1530 4.4152596781425215
1531 8.757200984608478
1532 6.677487720759766
1533 8.426831990418295
1534 6.577285637571317
1535 7.493869596318778
1536 8.7151200080925
1537 8.094008892062753
1538 5.831127652885081
1539 8.709532579215772
1540 8.174543577159849
1541 7.523370614423696
1542 8.05477107192397
1543 8.31464585548871
1544 8.574485878929147
1545 8.223869895699803
1546 4.884977122534998
1547 5.4466940

...

9204 6.109306553878856
9205 5.925801449973052
9206 7.0500866311617765
9207 7.3976109475464815
9208 6.057961609492115
9209 7.801258101388749
9210 7.493869596318778
9211 6.443315835118534
9212 6.3732589822241685
9213 6.157933580774859
9214 3.9166495911168075
9215 4.76950028127582
9216 7.671184512137573
9217 6.920059185010526
9218 4.878171957939991
9219 6.493653305451713
9220 8.671579904027597
9221 5.0089606168144964
9222 7.301039252524406
9223 8.233307958262657
9224 8.811472745889803
9225 7.480702853278462
9226 5.060426101537875
9227 8.31464585548871
9228 7.398684622324296
9229 7.170997223562014
9230 7.095065807028763
9231 7.475096634526175
9232 7.173825112577615
9233 6.256623555456826
9234 7.1814051722369925
9235 6.548256302136768
9236 7.31175211037078
9237 7.2603648940118175
9238 7.0500866311617765
9239 7.828314657173991
9240 7.170997223562014
9241 6.256623555456826
9242 5.2898411981419935
9243 8.753452412523746
9244 6.788244820250587
9245 8.358950719271759
9246 5.8629932327793535
9247

9580 8.022968937262847
9581 8.46061074997734
9582 3.396937997850927
9583 6.339533229884887
9584 6.606181060900379
9585 3.9166495911168075
9586 4.319273404069607
9587 7.900790082378434
9588 6.370760113378997
9589 3.183955536934048
9590 6.239461642959075
9591 4.547728129653585
9592 7.544492890288928
9593 4.823269134879355
9594 6.746195481627737
9595 5.5707483065472765
9596 7.696698229700935
9597 6.481250641368028
9598 7.760445779606027
9599 4.884977122534998
9600 7.218308566987025
9601 6.361917241451971
9602 8.743005258264253
9603 6.75912498649112
9604 6.1234614690910565
9605 6.127475169414972
9606 7.202030359518133
9607 4.395058287782974
9608 6.932987326135803
9609 3.167946225052173
9610 7.19431641656822
9611 7.493869596318778
9612 8.574485878929147
9613 5.7750166020415
9614 6.1693738810759235
9615 5.395914332073172
9616 8.174543577159849
9617 5.793133906306418
9618 6.168992330132397
9619 5.691791949528471
9620 5.376794142123453
9621 8.469379704137364
9622 8.43436991246704
9623 5.913085

9960 7.120553206667017
9961 6.653815670753763
9962 8.094008892062753
9963 7.176828794640382
9964 8.842186219376321
9965 8.233307958262657
9966 8.193933690074541
9967 7.1490024751244325
9968 6.843519109023448
9969 7.170997223562014
9970 6.590175893363125
9971 7.555622318778776
9972 4.319273404069607
9973 6.975487703374523
9974 6.1731344641759245
9975 5.771316867725653
9976 8.7151200080925
9977 5.759325715523379
9978 6.127475169414972
9979 6.732289493171827
9980 7.921906947449914
9981 4.884977122534998
9982 7.9139138493143415
9983 6.1731344641759245
9984 6.44913258358117
9985 7.737592628105482
9986 6.952734376604095
9987 6.813233939723765
9988 8.154949057424771
9989 2.405530567868847
9990 8.580480617777244
9991 8.001996457056773
9992 7.594723203639872
9993 8.074493292790843
9994 8.671579904027597
9995 5.884619275354042
9996 6.325430048473749
9997 6.181254371017418
9998 4.884977122534998
9999 6.361917241451971
10000 8.010034213468302
10001 6.764245708632437
10002 6.1234614690910565
10003 

10309 5.379115360527465
10310 8.671579904027597
10311 6.493653305451713
10312 6.256623555456826
10313 7.439325202919061
10314 4.1536709832001835
10315 4.941199293799766
10316 6.975487703374523
10317 6.978017625465581
10318 8.001996457056773
10319 8.747827810845564
10320 6.80968357898411
10321 6.935684111767527
10322 6.633970466631683
10323 6.014824585278642
10324 7.346453168347591
10325 5.270737082259082
10326 7.828314657173991
10327 8.656796740464273
10328 6.340092509269164
10329 8.233307958262657
10330 6.355865537958907
10331 8.574485878929147
10332 6.6039314074233415
10333 7.526966251345008
10334 8.525966352566687
10335 6.594243138351611
10336 6.580810311249267
10337 6.493653305451713
10338 8.358950719271759
10339 6.653557191851898
10340 6.797054857963599
10341 5.681146682020516
10342 7.784805899628435
10343 6.216138498726337
10344 7.675270956201749
10345 6.181254371017418
10346 4.884977122534998
10347 8.358950719271759
10348 8.709532579215772
10349 7.130630582676544
10350 8.6567967

10652 7.475096634526175
10653 6.746195481627737
10654 6.355865537958907
10655 8.709532579215772
10656 4.4152596781425215
10657 6.255858008511424
10658 6.636611724351833
10659 7.0500866311617765
10660 6.1731344641759245
10661 6.073246722162579
10662 5.95987631568777
10663 6.80968357898411
10664 6.916893730093862
10665 6.567779479406603
10666 3.0736611181364832
10667 8.585381211066728
10668 7.346453168347591
10669 1.3809619012215082
10670 8.469379704137364
10671 8.174543577159849
10672 8.8007305411162
10673 3.167946225052173
10674 5.883315873736841
10675 7.472890883466161
10676 7.9046655268312
10677 5.935867341951539
10678 7.901460184577196
10679 6.471278979163006
10680 8.656796740464273
10681 7.828314657173991
10682 6.717293132395666
10683 6.590116295563781
10684 8.671579904027597
10685 5.831127652885081
10686 6.427419698329425
10687 7.648306726574672
10688 7.648306726574672
10689 7.106682637574135
10690 8.053319101151267
10691 8.461328612234345
10692 6.998161370910088
10693 6.181732964

In [49]:
drug_test_data["base_score"] = pd.Series(rfr_predicted_y_test_data)
drug_test_data.head(5)

,patient_id,name_of_drug,review_by_patient,drug_approved_by_UIC,number_of_times_prescribed,use_case_for_drug,effectiveness_rating,base_score
0,163740,Mirtazapine,"""I&#039;ve tried a few antidepressants over th...",28-Feb-12,22,Depression,10,8.757201
1,39293,Contrave,"""Contrave combines drugs that were used for al...",5-Mar-17,35,Weight Loss,9,8.745037
2,208087,Zyclara,"""4 days in on first 2 weeks. Using on arms an...",3-Jul-14,13,Keratosis,4,6.162047
3,23295,Methadone,"""Ive been on Methadone for over ten years and ...",18-Oct-16,21,Opiate Withdrawal,7,4.564406
4,97013,Ambien,"""Ditto on rebound sleepless when discontinued....",13-Jan-15,44,Insomnia,2,7.596751


In [51]:
drug_test_data.to_csv("../dataset/mlresult.csv")

In [54]:
drug_test_data.loc[:, ["patient_id", "base_score"]].to_csv("../dataset/mlresult2.csv")

In [53]:
drug_test_data.loc[:, ["patient_id", "base_score"]].head(5)

,patient_id,base_score
0,163740,8.757201
1,39293,8.745037
2,208087,6.162047
3,23295,4.564406
4,97013,7.596751
